<a href="https://colab.research.google.com/github/taehyeonk/pre-onboarding_project/blob/jina/220314_sub3_pre%2Bsentence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import sys
import pandas as pd
import numpy as np 
import re
from typing import List, Dict
import math, copy, time
import random
import torch

In [2]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 3.8 MB 9.0 MB/s 
     |████████████████████████████████| 6.5 MB 53.9 MB/s 
     |████████████████████████████████| 596 kB 39.6 MB/s 
     |████████████████████████████████| 67 kB 4.5 MB/s 
     |████████████████████████████████| 895 kB 56.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 312 kB 9.6 MB/s 
     |████████████████████████████████| 212 kB 47.7 MB/s 
     |████████████████████████████████| 1.1 MB 41.5 MB/s 
     |████████████████████████████████| 134 kB 63.0 MB/s 
     |████████████████████████████████| 127 kB 64.9 MB/s 
     |████████████████████████████████| 144 kB 60.6 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 271 kB 53.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [30]:
!pip install sentence_transformers

     |████████████████████████████████| 79 kB 5.3 MB/s 
     |████████████████████████████████| 1.2 MB 20.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=49961a7f71a0f4379fa022391c52656d3cbef5d89d27fcb67e12f4f208047a00
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [14]:
# seed
seed = 7777
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [5]:
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla P100-PCIE-16GB
cuda


In [6]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [7]:
%cd /content/drive/MyDrive/Colab Notebooks/klue-sts-v1.1

/content/drive/MyDrive/Colab Notebooks/klue-sts-v1.1


In [8]:
df=pd.read_json('klue-sts-v1.1_train.json', orient='records')
df.drop(columns='annotations',inplace=True)
df.head()

,guid,source,sentence1,sentence2,labels
0,klue-sts-v1_train_00000,airbnb-rtt,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,"{'label': 3.7, 'real-label': 3.714285714285714..."
1,klue-sts-v1_train_00001,policy-sampled,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,"{'label': 0.0, 'real-label': 0.0, 'binary-labe..."
2,klue-sts-v1_train_00002,paraKQC-sampled,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,"{'label': 0.30000000000000004, 'real-label': 0..."
3,klue-sts-v1_train_00003,policy-sampled,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.","{'label': 0.6000000000000001, 'real-label': 0...."
4,klue-sts-v1_train_00004,airbnb-rtt,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,"{'label': 4.7, 'real-label': 4.714285714285714..."


In [43]:
def clean_text(texts):
    corpus = [] 
    for i in range(0, len(texts)):
        review = re.sub(r'\s+', ' ', str(texts[i]))  #과다한 띄어쓰기를 한번의 띄어쓰기로 변경
        review = review.lower() #영어 일단 소문자화
        review = re.sub(r'<[^>]+>','',review) #Html tags 제거
        review = re.sub(r'[^ A-Za-z0-9가-힣+]','',review) #특수문자 제거
        review = re.sub(r'\s+', ' ', review) #과다한 띄어쓰기를 한번의 띄어쓰기로 변경22
        review = re.sub(r"^\s+", '', review) #띄어쓰기로 시작하는 것 제거
        review = re.sub(r'\s+$', '', review) #띄어쓰기로 끝나는 것 제거
        corpus.append(review)
    return corpus
#review = re.sub(r'[-=+,#/\?:■^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', review) #remove punctuation

In [44]:
from torch.utils.data import DataLoader
from datasets import load_dataset
datasets = load_dataset("klue", "sts")

Reusing dataset klue (/root/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)


  0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
from datetime import datetime
train_batch_size = 32
num_epochs = 4
model_name='klue/roberta-large'
model_save_path = "output/training_klue_sts_" + model_name.replace("/", "-") + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [31]:
from sentence_transformers import SentenceTransformer, losses, models, util
embedding_model = models.Transformer(model_name)

Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [32]:
pooler = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)
model = SentenceTransformer(modules=[embedding_model, pooler])

In [39]:
type(datasets['train']['sentence1'])

list

In [46]:
train_samples = []
dev_samples = []
from sentence_transformers import InputExample
# KLUE STS 내 훈련, 검증 데이터 예제 변환
for phase in ["train", "validation"]:
    examples = datasets[phase]
    for i in range(len(examples)):
        examples["sentence1"][i]=clean_text(examples["sentence1"][i])
        examples["sentence2"][i]=clean_text(examples["sentence2"][i])


    for example in examples:       
        score = float(example["labels"]["label"]) / 5.0  # 0.0 ~ 1.0 스케일로 유사도 정규화

        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]], 
            label=score,
        )

        if phase == "validation":
            dev_samples.append(inp_example)
        else:
            train_samples.append(inp_example)

In [49]:
train_samples[120].texts, train_samples[120].label

(['이들 사이트들은 한시적으로 5월 말까지 일반 학생들도 스마트폰으로 데이터 사용량에 대한 부담 없이 이용이 가능하도록 ‘제로레이팅(특정사이트 데이터 무과금)’으로 지원한다.',
  '이들 사이트는 일시적으로 "제로 레이트"(특정 사이트 데이터의 경우 무료)를 지원하여 5월 말까지 일반 학생들이 데이터 사용에 대한 부담 없이 스마트폰을 사용할 수 있도록 합니다.'],
 0.86)

In [50]:
train_dataloader = DataLoader(
    train_samples,
    shuffle=True,
    batch_size=train_batch_size,
)
train_loss = losses.CosineSimilarityLoss(model=model)

In [51]:
import logging
from sentence_transformers import  LoggingHandler
logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1)  # 10% of train data for warm-up
logging.info(f"Warmup-steps: {warmup_steps}")

2022-03-14 13:03:33 - Warmup-steps: 146


In [52]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    dev_samples,
    name="sts-dev",
)

In [53]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=num_epochs,
    evaluation_steps=1000,
    warmup_steps=warmup_steps,
    output_path=model_save_path,
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2022-03-14 13:10:28 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 0:
2022-03-14 13:10:31 - Cosine-Similarity :	Pearson: 0.8777	Spearman: 0.8721
2022-03-14 13:10:31 - Manhattan-Distance:	Pearson: 0.8787	Spearman: 0.8741
2022-03-14 13:10:31 - Euclidean-Distance:	Pearson: 0.8788	Spearman: 0.8740
2022-03-14 13:10:31 - Dot-Product-Similarity:	Pearson: 0.8605	Spearman: 0.8493
2022-03-14 13:10:31 - Save model to output/training_klue_sts_klue-roberta-large-2022-03-14_12-27-48


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2022-03-14 13:16:20 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 1:
2022-03-14 13:16:23 - Cosine-Similarity :	Pearson: 0.8839	Spearman: 0.8824
2022-03-14 13:16:23 - Manhattan-Distance:	Pearson: 0.8823	Spearman: 0.8800
2022-03-14 13:16:23 - Euclidean-Distance:	Pearson: 0.8821	Spearman: 0.8797
2022-03-14 13:16:23 - Dot-Product-Similarity:	Pearson: 0.8687	Spearman: 0.8616
2022-03-14 13:16:23 - Save model to output/training_klue_sts_klue-roberta-large-2022-03-14_12-27-48


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2022-03-14 13:22:14 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 2:
2022-03-14 13:22:17 - Cosine-Similarity :	Pearson: 0.8864	Spearman: 0.8844
2022-03-14 13:22:17 - Manhattan-Distance:	Pearson: 0.8868	Spearman: 0.8837
2022-03-14 13:22:17 - Euclidean-Distance:	Pearson: 0.8862	Spearman: 0.8832
2022-03-14 13:22:17 - Dot-Product-Similarity:	Pearson: 0.8722	Spearman: 0.8665
2022-03-14 13:22:17 - Save model to output/training_klue_sts_klue-roberta-large-2022-03-14_12-27-48


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2022-03-14 13:28:07 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 3:
2022-03-14 13:28:11 - Cosine-Similarity :	Pearson: 0.8873	Spearman: 0.8858
2022-03-14 13:28:11 - Manhattan-Distance:	Pearson: 0.8875	Spearman: 0.8847
2022-03-14 13:28:11 - Euclidean-Distance:	Pearson: 0.8868	Spearman: 0.8838
2022-03-14 13:28:11 - Dot-Product-Similarity:	Pearson: 0.8732	Spearman: 0.8675
2022-03-14 13:28:11 - Save model to output/training_klue_sts_klue-roberta-large-2022-03-14_12-27-48


In [64]:
testsets = load_dataset("kor_nlu", "sts")
test_samples = []
# KorSTS 내 테스트 데이터 예제 변환
for example in testsets["test"]:
    score = float(example["score"]) / 5.0

    if example["sentence1"] and example["sentence2"]:
        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]],
            label=score,
        )

    test_samples.append(inp_example)
model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')

2022-03-14 13:36:01 - Reusing dataset kor_nlu (/root/.cache/huggingface/datasets/kor_nlu/sts/1.0.0/4facbba77df60b0658056ced2052633e681a50187b9428bd5752ebd59d332ba8)


  0%|          | 0/3 [00:00<?, ?it/s]

2022-03-14 13:36:01 - Load pretrained SentenceTransformer: output/training_klue_sts_klue-roberta-large-2022-03-14_12-27-48
2022-03-14 13:36:06 - Use pytorch device: cuda


In [65]:
test_evaluator(model, output_path=model_save_path)

2022-03-14 13:36:26 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-test dataset:
2022-03-14 13:36:35 - Cosine-Similarity :	Pearson: 0.7840	Spearman: 0.7752
2022-03-14 13:36:35 - Manhattan-Distance:	Pearson: 0.7651	Spearman: 0.7709
2022-03-14 13:36:35 - Euclidean-Distance:	Pearson: 0.7649	Spearman: 0.7706
2022-03-14 13:36:35 - Dot-Product-Similarity:	Pearson: 0.7648	Spearman: 0.7558


0.7752107416848436